## BLOCKMODEL GEOLOGI STEP BY STEP DENGAN DATAMINE STUDIO RM DAN PYTHON


Salah satu cara untuk mempercepat proses pembuatan blokmodel dengan Datamine adalah menggunakan script atau macro. Tetapi hal ini terkadang sulit dimengerti oleh geologis pemula, karena tidak mengetahui proses yang terjadi di balik script atau macro tersebut. 
Jupyter notebook / python akan membantu proses automatisasi sekaligus memperlihatkan secara transparan proses yang terjadi.
Berikut beberapa keuntungan menggunakan Jupyter notebook python untuk proses pembuatan blokmodel :
* Membantu geologis pemula untuk belajar step by step pembuatan blokmodel
* Membantu tracing pembuatan blokmodel / logbook, sehingga mempermudah pekerjaan peer review blokmodel.
* Bisa dijadikan bukti otentik jika suatu saat diaudit atau dispute/perkara pengadilan.
* Mempercepat proses pembuatan blokmodel.
Pada notebook ini, penggunaan python melalui package 'dmstudio' yang dibuat oleh Sean Horan, package bisa di ambil di https://github.com/seanhoran/dmstudio.

##### IDE menggunakan Anaconda.
##### Selvi Yuminti, Desember 2021.
##### Version 1.0

In [1]:
import pandas as pd

In [2]:
import win32com.client as client 

In [3]:
oScript = client.Dispatch("Datamine.StudioRM.Application")

In [4]:
import dmstudio.dmcommands as dmcommands  # panggil package dmstudio

pyrpa module not found, only available to SLR employees


In [5]:
dm = dmcommands.init() # memulai koneksi ke Datamine Studio RM

## 1. Pembuatan drillhole 3D
Untuk import file assay dan collar tidak dimasukkan ke dalam step, begitu juga proses ETL dan EDA. Jadi asumsi di sini adalah file assay.dm dan collar.dm sudah ada di direktori kerja.

In [6]:
dm.holes3d(
    collar_i='collar',
    survey_i='optional',
    samples_i=['assay'],
    out_o='holes1',
    holesmry_o='hls_smry',
    errors_o='hls_err',
    bhid_f='BHID',
    xcollar_f='XCOLLAR',
    ycollar_f='YCOLLAR',
    zcollar_f='ZCOLLAR',
    from_f='FROM',
    to_f='TO',
    at_f='optional',
    brg_f='optional',
    dip_f='optional',
    survsmth_p=1,
    endpoint_p=0,
    dipmeth_p=-1,
    print_p=0,
    retrieval='optional',
)


holes3d  &collar=collar &sample1=assay  &out=holes1 &holesmry=hls_smry &errors=hls_err *bhid=BHID *xcollar=XCOLLAR *ycollar=YCOLLAR *zcollar=ZCOLLAR *from=FROM *to=TO @survsmth=1 @endpoint=0 @dipmeth=-1 @print=0


## 2. Kalkulasi LDW
Beberapa assay, sudah berisi data LDW1 dan LDW6, jika belum ada, run perintah di bawah.

In [7]:
dm.extra(
    in_i='holes1',
    out_o='holes',
    print_p='optional',
    approx_p='optional',
    expression=['LDW1=DW1/LENGTH','LDW6=DW6/LENGTH'],
    retrieval='optional',
)

## 3. Membuat layer points dari drillhole

In [8]:
dm.seldel(
    in_i='holes',
    fieldlst_i='optional',
    out_o='t_sel',
    f1_f='NITOT',
    f2_f='COTOT',
    f3_f='FETOT',
    f4_f='SIO2TOT',
    f5_f='MGOTOT',
    f6_f='CRTOT',
    f7_f='ALTOT',
    f8_f='MNTOT',
    f9_f='CATOT',
    f10_f='optional',
    f11_f='optional',
    f12_f='optional',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    retrieval='optional',
)

In [9]:
dm.copy(in_i='t_sel', out_o='t_holes', retrieval='optional')

### Membuat LIM top

In [10]:
dm.extra(
    in_i='t_holes',
    out_o='t_2',
    print_p='optional',
    approx_p='optional',
    expression=['FIRST;N=0 IF(PREV(BHID)!=BHID) FIRST=1 END', 'IF(FIRST==1) ZPT=Z+((LENGTH/2)*SIN(B0)) END', 'IF(FIRST==1) XPT=X-((LENGTH/2)*COS(B0))*SIN(A0) END', 'IF(FIRST==1) YPT=Y-((LENGTH/2)*COS(B0))*COS(A0) END'],
    retrieval='optional',
)

In [11]:
dm.extra(
    in_i='t_2',
    out_o='t_first',
    print_p='optional',
    approx_p='optional',
    expression=['COLOUR=1','SYMBOL=201'],
    retrieval='FIRST=1',
)

In [12]:
dm.selcop(
    in_i='t_first',
    fieldlst_i='optional',
    out_o='ltoppts',
    f1_f='XPT',
    f2_f='YPT',
    f3_f='ZPT',
    f4_f='COLOUR',
    f5_f='SYMBOL',
    f6_f='optional',
    f7_f='optional',
    f8_f='optional',
    f9_f='optional',
    f10_f='optional',
    f11_f='optional',
    f12_f='optional',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    retrieval='optional',
)

### Membuat BRK Bottom = EOH in Core DHS

In [13]:
dm.extra(
    in_i='t_holes',
    out_o='t_3',
    print_p='optional',
    approx_p='optional',
    expression=['LAST;N=0 IF(NEXT(BHID)!=BHID) LAST=1 END', 'IF(LAST==1) ZPT=Z-((LENGTH/2)*SIN(B0)) END', 'IF(LAST==1) XPT=X+((LENGTH/2)*COS(B0))*SIN(A0) END', 'IF(LAST==1) YPT=Y+((LENGTH/2)*COS(B0))*COS(A0) END'],
    retrieval='optional',
)

In [14]:
dm.extra(
    in_i='t_3',
    out_o='t_4',
    print_p='optional',
    approx_p='optional',
    expression=['COLOUR=4','SYMBOL=201'],
    retrieval='LAST=1',
)

In [15]:
dm.selcop(
    in_i='t_4',
    fieldlst_i='optional',
    out_o='bbotpts',
    f1_f='XPT',
    f2_f='YPT',
    f3_f='ZPT',
    f4_f='COLOUR',
    f5_f='SYMBOL',
    f6_f='optional',
    f7_f='optional',
    f8_f='optional',
    f9_f='optional',
    f10_f='optional',
    f11_f='optional',
    f12_f='optional',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    retrieval='optional',
)

### Check file untuk colocated holes dan laporkan dalam file csv

In [16]:
dm.sortx(
    in_i='t_first',
    out_o='t_5',
    key1_f='BHID',
    key2_f='XPT',
    key3_f='YPT',
    key4_f='optional',
    key5_f='optional',
    key6_f='optional',
    key7_f='optional',
    key8_f='optional',
    key9_f='optional',
    key10_f='optional',
    order_p=1,
    keysfrst_p=1,
    roworder_p=1,
    retrieval='optional',
)

In [17]:
dm.selcop(
    in_i='t_5',
    fieldlst_i='optional',
    out_o='t_6',
    f1_f='BHID',
    f2_f='XPT',
    f3_f='YPT',
    f4_f='optional',
    f5_f='optional',
    f6_f='optional',
    f7_f='optional',
    f8_f='optional',
    f9_f='optional',
    f10_f='optional',
    f11_f='optional',
    f12_f='optional',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    retrieval='optional',
)

#### Terdapat bugs pada dm.count di module dmcommands.py , sehingga command ini belum bisa digunakan, 
line 5361 if key1_f == "required":
seharusnya
if keys_f[0] == "required":


dm.count(
    in_i='t_6',
    out_o='t_bhc',
    keys_f=['BHID','XPT','YPT'],
    retrieval='optional',
)

Karena bugs pada dm.count belum diupdate oleh ownernya, kita akan memakai modul win32client dan oScript untuk langsung mendefinisikan perintah 'count' dari Datamine

In [18]:
command_count = "count &in=t_6 &out=t_bhc *key1=BHID *key2=XPT *key3=YPT"
command_count

'count &in=t_6 &out=t_bhc *key1=BHID *key2=XPT *key3=YPT'

In [19]:
oScript.Parsecommand(command_count)

In [20]:
dm.sortx(
    in_i='t_bhc',
    out_o='t_7',
    key1_f='XPT',
    key2_f='YPT',
    key3_f='BHID',
    key4_f='optional',
    key5_f='optional',
    key6_f='optional',
    key7_f='optional',
    key8_f='optional',
    key9_f='optional',
    key10_f='optional',
    order_p=1,
    keysfrst_p=1,
    roworder_p=1,
    retrieval='optional',
)

In [21]:
dm.copynr(
    in_i='t_7',
    out_o='t_7a',
    base_p='optional',
    incrment_p='optional',
    retrieval='optional',
)

In [22]:
dm.extra(
    in_i='t_7a',
    out_o='t_8',
    print_p='optional',
    approx_p='optional',
    expression=['XDIFF=(XPT-PREV(XPT)) YDIFF=(YPT-PREV(YPT)) XDIFF2=ABS(PREV(XPT)-XPT) YDIFF2=ABS(PREV(YPT)-YPT) DIFFLT1=(XDIFF <=1 AND YDIFF <= 1 AND RECORDNO !=1) DIFF2LT1=(XDIFF2 <= 1 AND YDIFF2 <= 1 AND RECORDNO != 1)', 'IF(DIFFLT1==1 AND DIFF2LT1==1) SAME_XY=1 END'],
    retrieval='optional',
)

In [23]:
dm.extra(
    in_i='t_8',
    out_o='t_8a',
    print_p='optional',
    approx_p='optional',
    expression=['XDIFF=(XPT-NEXT(XPT)) YDIFF=(YPT-NEXT(YPT)) XDIFF2=ABS(NEXT(XPT)-XPT) YDIFF2=ABS(NEXT(YPT)-YPT) DIFFLT1=(XDIFF <=1 AND YDIFF <= 1) DIFF2LT1=(XDIFF2 <= 1 AND YDIFF2 <= 1)', 'IF(DIFFLT1==1 AND DIFF2LT1==1) SAME_XY=1 END'],
    retrieval='optional',
)

In [24]:
dm.selcop(
    in_i='t_8a',
    fieldlst_i='optional',
    out_o='t_9',
    f1_f='BHID',
    f2_f='XPT',
    f3_f='YPT',
    f4_f='optional',
    f5_f='optional',
    f6_f='optional',
    f7_f='optional',
    f8_f='optional',
    f9_f='optional',
    f10_f='optional',
    f11_f='optional',
    f12_f='optional',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    retrieval='SAME_XY=1',
)

In [25]:
dm.extra(
    in_i='t_9',
    out_o='t_9a',
    print_p='optional',
    approx_p='optional',
    expression=['X=XPT','Y=YPT'],
    retrieval='optional',
)

In [26]:
dm.sortx(
    in_i='t_9a',
    out_o='colocatd',
    key1_f='BHID',
    key2_f='X',
    key3_f='Y',
    key4_f='optional',
    key5_f='optional',
    key6_f='optional',
    key7_f='optional',
    key8_f='optional',
    key9_f='optional',
    key10_f='optional',
    order_p=1,
    keysfrst_p=1,
    roworder_p=1,
    retrieval='optional',
)

In [27]:
dm.output(
    in_i='colocatd',
    fieldlst_i='optional',
    f1_f='optional',
    f2_f='optional',
    f3_f='optional',
    f4_f='optional',
    f5_f='optional',
    f6_f='optional',
    f7_f='optional',
    f8_f='optional',
    f9_f='optional',
    f10_f='optional',
    f11_f='optional',
    f12_f='optional',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    csv_p=1,
    nodd_p=0,
    dplace_p=-1,
    implicit_p=0,
    csv_o='same_collar_location_drillholes.csv',
    retrieval='optional',
)

In [28]:
dm.delete(in_i='colocatd', confirm_p='optional', retrieval='optional')

### Cek interval gap pada file holes atau assay dan buat laporan csv-nya

In [29]:
dm.sortx(
    in_i='t_holes',
    out_o='t_10',
    key1_f='BHID',
    key2_f='FROM',
    key3_f='TO',
    key4_f='optional',
    key5_f='optional',
    key6_f='optional',
    key7_f='optional',
    key8_f='optional',
    key9_f='optional',
    key10_f='optional',
    order_p=1,
    keysfrst_p=1,
    roworder_p=1,
    retrieval='optional',
)

In [30]:
dm.selcop(
    in_i='t_10',
    fieldlst_i='optional',
    out_o='t_10a',
    f1_f='BHID',
    f2_f='FROM',
    f3_f='TO',
    f4_f='optional',
    f5_f='optional',
    f6_f='optional',
    f7_f='optional',
    f8_f='optional',
    f9_f='optional',
    f10_f='optional',
    f11_f='optional',
    f12_f='optional',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    retrieval='optional',
)

In [31]:
dm.extra(
    in_i='t_10a',
    out_o='t_11',
    print_p='optional',
    approx_p='optional',
    expression=['BHPREV=(PREV(BHID)==BHID) BHNEXT=(NEXT(BHID)==BHID) BADGAP=0 FRMTODIF=ABS(FROM-PREV(TO))', 'IF(BHPREV==1 AND FRMTODIF>0) BADGAP=1 END'],
    retrieval='optional',
)

In [32]:
dm.extra(
    in_i='t_11',
    out_o='t_12',
    print_p='optional',
    approx_p='optional',
    expression=['FRMTODF2=ABS(TO-NEXT(FROM))', 'IF(BHNEXT==1 AND FRMTODF2>0) BADGAP=1 END'],
    retrieval='optional',
)

In [33]:
dm.selcop(
    in_i='t_12',
    fieldlst_i='optional',
    out_o='badgap',
    f1_f='BHID',
    f2_f='FROM',
    f3_f='TO',
    f4_f='optional',
    f5_f='optional',
    f6_f='optional',
    f7_f='optional',
    f8_f='optional',
    f9_f='optional',
    f10_f='optional',
    f11_f='optional',
    f12_f='optional',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    retrieval='BADGAP=1',
)

In [34]:
dm.output(
    in_i='badgap',
    fieldlst_i='optional',
    f1_f='optional',
    f2_f='optional',
    f3_f='optional',
    f4_f='optional',
    f5_f='optional',
    f6_f='optional',
    f7_f='optional',
    f8_f='optional',
    f9_f='optional',
    f10_f='optional',
    f11_f='optional',
    f12_f='optional',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    csv_p=1,
    nodd_p=0,
    dplace_p=-1,
    implicit_p=0,
    csv_o='missing_sample_intervals.csv',
    retrieval='optional',
)

Bugs pada dm.count (sudah dijelaskan di atas)

dm.count(
    in_i='badgap',
    out_o='t_1',
    keys_f=['BHID'],
    retrieval='optional',
)

In [35]:
command_count = "count &in=badgap &out=t_1 *key1=BHID"
command_count

'count &in=badgap &out=t_1 *key1=BHID'

In [36]:
oScript.Parsecommand(command_count)

In [37]:
dm.copy(in_i='badgap', out_o='t_bad', retrieval='optional')

In [38]:
dm.delete(in_i='badgap', confirm_p='optional', retrieval='optional')

if t_bad > 0:
    print("There are + t_1 drillholes in + HOLES + with sample interval gaps. \n\n Look at file MISSING_SAMPLE_INTERVALS.CSV for a list of the samples \n above and below the gap. \n Fix Database and then rerun script.")

Cari : Jika t_bad > 0, print : There are + t_1 drillholes in + HOLES + with sample interval gaps. \n\n Look at file MISSING_SAMPLE_INTERVALS.CSV for a list of the samples \n above and below the gap. \n Fix Database and then rerun script.

### Cek >3 layer per drillhole dan buat laporan csv-nya
### Buat single sample untuk masing-masing unit litologi

In [39]:
dm.compdh(
    in_i='t_holes',
    out_o='t_13',
    bhid_f='BHID',
    from_f='FROM',
    to_f='TO',
    density_f='optional',
    coreloss_f='optional',
    corerec_f='optional',
    zone_f='LYR',
    interval_p='100',
    mingap_p='0.001',
    maxgap_p=0.001,
    mincomp_p='0.1',
    loss_p='optional',
    start_p=0,
    mode_p=0,
    print_p=0,
    retrieval='optional',
)

In [40]:
dm.sortx(
    in_i='t_13',
    out_o='t_14',
    key1_f='BHID',
    key2_f='LYR',
    key3_f='optional',
    key4_f='optional',
    key5_f='optional',
    key6_f='optional',
    key7_f='optional',
    key8_f='optional',
    key9_f='optional',
    key10_f='optional',
    order_p=1,
    keysfrst_p=1,
    roworder_p=1,
    retrieval='optional',
)

In [41]:
dm.selcop(
    in_i='t_14',
    fieldlst_i='optional',
    out_o='t_14a',
    f1_f='BHID',
    f2_f='LYR',
    f3_f='optional',
    f4_f='optional',
    f5_f='optional',
    f6_f='optional',
    f7_f='optional',
    f8_f='optional',
    f9_f='optional',
    f10_f='optional',
    f11_f='optional',
    f12_f='optional',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    retrieval='optional',
)

In [42]:
command_count = "count &in=t_14a &out=t_15 *key1=BHID *key2=LYR"
command_count

'count &in=t_14a &out=t_15 *key1=BHID *key2=LYR'

In [43]:
oScript.Parsecommand(command_count)

#### Cek >1 layer untuk tiap domain (misal LIM antara upper dan lower SAP) 

In [44]:
dm.copy(in_i='t_15', out_o='gt1layer', retrieval='COUNT>1.5')

In [45]:
dm.output(
    in_i='gt1layer',
    fieldlst_i='optional',
    f1_f='optional',
    f2_f='optional',
    f3_f='optional',
    f4_f='optional',
    f5_f='optional',
    f6_f='optional',
    f7_f='optional',
    f8_f='optional',
    f9_f='optional',
    f10_f='optional',
    f11_f='optional',
    f12_f='optional',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    csv_p=1,
    nodd_p=0,
    dplace_p=-1,
    implicit_p=0,
    csv_o='gt1layer.csv',
    retrieval='optional',
)

In [46]:
command_count = "count &in=gt1layer &out=t_16 *key1=BHID"
command_count

'count &in=gt1layer &out=t_16 *key1=BHID'

In [47]:
oScript.Parsecommand(command_count)

In [48]:
dm.copy(in_i='gt1layer', out_o='t_gt', retrieval='COUNT>1.5')

In [49]:
dm.delete(in_i='gt1layer', confirm_p='optional', retrieval='optional')

If t_gt > 0, if(!confirm("There are " + recs4 + " drillholes in " + tbHOLES.value + " with > 3 " + tbFIELD.value + " values." +
			"\n\nLook at file GT1LAYER.CSV for a list of the holes" +
			"\n\nClick on Cancel to go fix Database and then rerun script.\nOr Click on OK to continue.")){
	dm.command("delete-file 't_*' 'yes'");

### Membuat LIM Bottom
### Gunakan filter untuk memilih field value yang merupakan lokasi bottom yang kita inginkan

In [50]:
dm.selcop(
    in_i='t_holes',
    fieldlst_i='optional',
    out_o='t_17',
    f1_f='BHID',
    f2_f='FROM',
    f3_f='TO',
    f4_f='X',
    f5_f='Y',
    f6_f='Z',
    f7_f='LENGTH',
    f8_f='A0',
    f9_f='B0',
    f10_f='C0',
    f11_f='RADIUS',
    f12_f='LYR',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    retrieval="LYR='LIM'",
)

In [51]:
dm.extra(
    in_i='t_17',
    out_o='t_18',
    print_p='optional',
    approx_p='optional',
    expression=['LAST;N=0 IF(NEXT(BHID)!=BHID) LAST=1 END', 'IF(LAST==1) ZPT=Z-((LENGTH/2)*SIN(B0)) END', 'IF(LAST==1) XPT=X+((LENGTH/2)*COS(B0))*SIN(A0) END', 'IF(LAST==1) YPT=Y+((LENGTH/2)*COS(B0))*COS(A0) END'],
    retrieval='optional',
)

In [52]:
dm.extra(
    in_i='t_18',
    out_o='t_19',
    print_p='optional',
    approx_p='optional',
    expression=['COLOUR=2', 'SYMBOL=201', 'ERASE(X,Y,Z,FROM,TO)', 'ERASE(A0,B0,C0,RADIUS)'],
    retrieval='LAST=1',
)

In [53]:
dm.selcop(
    in_i='t_19',
    fieldlst_i='optional',
    out_o='t_20',
    f1_f='XPT',
    f2_f='YPT',
    f3_f='ZPT',
    f4_f='COLOUR',
    f5_f='SYMBOL',
    f6_f='optional',
    f7_f='optional',
    f8_f='optional',
    f9_f='optional',
    f10_f='optional',
    f11_f='optional',
    f12_f='optional',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    retrieval='optional',
)

In [54]:
dm.sortx(
    in_i='ltoppts',
    out_o='t_limtop',
    key1_f='XPT',
    key2_f='YPT',
    key3_f='ZPT',
    key4_f='optional',
    key5_f='optional',
    key6_f='optional',
    key7_f='optional',
    key8_f='optional',
    key9_f='optional',
    key10_f='optional',
    order_p=1,
    keysfrst_p=1,
    roworder_p=1,
    retrieval='optional',
)

In [55]:
dm.sortx(
    in_i='t_20',
    out_o='t_limbot',
    key1_f='XPT',
    key2_f='YPT',
    key3_f='ZPT',
    key4_f='optional',
    key5_f='optional',
    key6_f='optional',
    key7_f='optional',
    key8_f='optional',
    key9_f='optional',
    key10_f='optional',
    order_p=1,
    keysfrst_p=1,
    roworder_p=1,
    retrieval='optional',
)

### Ganti semua LIM Bottom points yang hilang dengan LIM Top points

In [56]:
dm.diffrn(
    in1_i='t_limtop',
    in2_i='t_limbot',
    out_o='t_dflim',
    keys_f=['XPT','YPT'],
    retrieval='optional',
)

In [57]:
dm.join(
    in1_i='t_limbot',
    in2_i='t_dflim',
    out_o='t_21',
    keys_f=['XPT','YPT'],
    subsetr_p=0,
    subsetf_p=0,
    cartjoin_p=0,
    print_p=0,
    retrieval='optional',
)

In [58]:
dm.selcop(
    in_i='t_21',
    fieldlst_i='optional',
    out_o='t_22',
    f1_f='XPT',
    f2_f='YPT',
    f3_f='ZPT',
    f4_f='COLOUR',
    f5_f='SYMBOL',
    f6_f='optional',
    f7_f='optional',
    f8_f='optional',
    f9_f='optional',
    f10_f='optional',
    f11_f='optional',
    f12_f='optional',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    retrieval='optional',
)

In [59]:
dm.extra(
    in_i='t_22',
    out_o='lbotpts',
    print_p='optional',
    approx_p='optional',
    expression=['COLOUR=2','SYMBOL=201',],
    retrieval='optional',
)

In [60]:
dm.delete(in_i='t_3', confirm_p='optional', retrieval='optional')

### Membuat SAP Bottom
### Gunakan filter untuk memilih field value untuk mendapatkan lokasi bottom yang diinginkan

In [61]:
dm.selcop(
    in_i='t_holes',
    fieldlst_i='optional',
    out_o='t_22',
    f1_f='BHID',
    f2_f='FROM',
    f3_f='TO',
    f4_f='X',
    f5_f='Y',
    f6_f='Z',
    f7_f='LENGTH',
    f8_f='A0',
    f9_f='B0',
    f10_f='C0',
    f11_f='RADIUS',
    f12_f='LYR',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    retrieval="LYR='SAP'",
)

In [62]:
dm.extra(
    in_i='t_22',
    out_o='t_23',
    print_p='optional',
    approx_p='optional',
    expression=['LAST;N=0 IF(NEXT(BHID)!=BHID) LAST=1 END', 'IF(LAST==1) ZPT=Z-((LENGTH/2)*SIN(B0)) END', 'IF(LAST==1) XPT=X+((LENGTH/2)*COS(B0))*SIN(A0) END', 'IF(LAST==1) YPT=Y+((LENGTH/2)*COS(B0))*COS(A0) END'],
    retrieval='optional',
)

In [63]:
dm.extra(
    in_i='t_23',
    out_o='t_24',
    print_p='optional',
    approx_p='optional',
    expression=['COLOUR=3','SYMBOL=212','ERASE(X,Y,Z,FROM,TO)'],
    retrieval='LAST=1',
)

In [64]:
dm.selcop(
    in_i='t_24',
    fieldlst_i='optional',
    out_o='sbotpts',
    f1_f='XPT',
    f2_f='YPT',
    f3_f='ZPT',
    f4_f='COLOUR',
    f5_f='SYMBOL',
    f6_f='optional',
    f7_f='optional',
    f8_f='optional',
    f9_f='optional',
    f10_f='optional',
    f11_f='optional',
    f12_f='optional',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    retrieval='optional',
)

#### Ganti points SAP yang hilang dengan points LIM bottom / Replace any missing SAP points with Bottom LIM points

In [65]:
dm.sortx(
    in_i='lbotpts',
    out_o='t_lstlim',
    key1_f='XPT',
    key2_f='YPT',
    key3_f='ZPT',
    key4_f='optional',
    key5_f='optional',
    key6_f='optional',
    key7_f='optional',
    key8_f='optional',
    key9_f='optional',
    key10_f='optional',
    order_p=1,
    keysfrst_p=1,
    roworder_p=1,
    retrieval='optional',
)

In [66]:
dm.sortx(
    in_i='sbotpts',
    out_o='t_lstsap',
    key1_f='XPT',
    key2_f='YPT',
    key3_f='ZPT',
    key4_f='optional',
    key5_f='optional',
    key6_f='optional',
    key7_f='optional',
    key8_f='optional',
    key9_f='optional',
    key10_f='optional',
    order_p=1,
    keysfrst_p=1,
    roworder_p=1,
    retrieval='optional',
)

In [67]:
dm.diffrn(
    in1_i='t_lstlim',
    in2_i='t_lstsap',
    out_o='t_dfsap',
    keys_f=['XPT','YPT'],
    retrieval='optional',
)

In [68]:
dm.join(
    in1_i='t_lstsap',
    in2_i='t_dfsap',
    out_o='t_25',
    keys_f=['XPT','YPT'],
    subsetr_p=0,
    subsetf_p=0,
    cartjoin_p=0,
    print_p=0,
    retrieval='optional',
)

In [69]:
dm.selcop(
    in_i='t_25',
    fieldlst_i='optional',
    out_o='t_26',
    f1_f='XPT',
    f2_f='YPT',
    f3_f='ZPT',
    f4_f='COLOUR',
    f5_f='SYMBOL',
    f6_f='optional',
    f7_f='optional',
    f8_f='optional',
    f9_f='optional',
    f10_f='optional',
    f11_f='optional',
    f12_f='optional',
    f13_f='optional',
    f14_f='optional',
    f15_f='optional',
    f16_f='optional',
    f17_f='optional',
    f18_f='optional',
    f19_f='optional',
    f20_f='optional',
    f21_f='optional',
    f22_f='optional',
    f23_f='optional',
    f24_f='optional',
    f25_f='optional',
    fieldnam_f='optional',
    retrieval='optional',
)

In [70]:
dm.extra(
    in_i='t_26',
    out_o='sbotpts',
    print_p='optional',
    approx_p='optional',
    expression=['COLOUR=3','SYMBOL=212'],
    retrieval='optional',
)

In [71]:
delete_file = "delete-file 't_*' 'yes'"
delete_file 

"delete-file 't_*' 'yes'"

In [72]:
oScript.Parsecommand(delete_file)

In [73]:
print("MEMBUAT POINTS SELESAI")

MEMBUAT POINTS SELESAI
